In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
import tensorflow as tf

In [3]:
business_path = '/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_business.json'
review_path = '/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_review.json'

In [4]:
# Load business data
business_df = pd.read_json(business_path, lines=True)
# Load review data
review_df = pd.read_json(review_path, lines=True, nrows=100000)

# Display basic information about the loaded datasets
print("Business DataFrame Info:")
print(business_df.info())

print("\nReview DataFrame Info:")
print(review_df.info())

# Sample of Business DataFrame
print("\nSample of Business DataFrame:")
print(business_df.head())

# Sample of Review DataFrame
print("\nSample of Review DataFrame:")
print(review_df.head())

Business DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB
None

Review DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 

In [5]:
# Step 3: Preprocess the Dataset
# Example of basic preprocessing (you may need to customize based on your specific needs)
def preprocess_text(text):
    # Example: convert text to lowercase
    text = text.lower()
    # Add more preprocessing steps as needed
    return text

# Apply text preprocessing to the 'text' column in the review DataFrame
review_df['text'] = review_df['text'].apply(preprocess_text)

# Handle missing values if any
review_df.dropna(inplace=True)

# Remove duplicate rows based on the 'text' column
review_df.drop_duplicates(subset=['text'], inplace=True)

# Assuming sentiment is positive for stars greater than or equal to 4, and negative otherwise
review_df['label'] = (review_df['stars'] >= 4).astype(int)

# Display the preprocessed DataFrame
print("\nSample of Preprocessed Review DataFrame:")
print(review_df.head())




Sample of Preprocessed Review DataFrame:
                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1      0     1   
4      4       1      0     1   

                                                text                date  \
0  if you decide to eat here, just be aware it is... 2018-07-07 22:09:11   
1  i've taken a lot of spin classes over the year... 2012-01-03 15:28:18   
2  family diner. had the buffet. eclectic assortm... 2014-02-05 20:3

In [6]:
# Save the preprocessed review DataFrame to a CSV file
preprocessed_csv_path = '/content/drive/MyDrive/yelp_dataset/preprocessed_reviews.csv'
review_df.to_csv(preprocessed_csv_path, index=False)
print(f"Preprocessed dataset saved to {preprocessed_csv_path}")



Preprocessed dataset saved to /content/drive/MyDrive/yelp_dataset/preprocessed_reviews.csv


In [7]:
# Split the dataset into training (80%) and test (20%) sets
train_data, test_data = train_test_split(review_df, test_size=0.2, random_state=42)

# Load your preprocessed Yelp Reviews dataset
data = pd.read_csv('/content/drive/MyDrive/yelp_dataset/preprocessed_reviews.csv')

# Assume you have a 'label' column in your dataset for sentiment (0 or 1)
labels = data['label'].tolist()



In [8]:
# Load pre-trained BERT model and tokenizer
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
# Tokenize and preprocess the training data
train_reviews = train_data['text'].tolist()
train_labels = train_data['label'].tolist()
train_inputs = tokenizer(train_reviews, padding=True, truncation=True, return_tensors='tf', max_length=128)  # Adjust max_length
train_inputs_numpy = {
    'input_ids': np.array(train_inputs['input_ids']),
    'attention_mask': np.array(train_inputs['attention_mask'])
}
train_labels = tf.convert_to_tensor(train_labels)

# Tokenize and preprocess the test data
test_reviews = test_data['text'].tolist()
test_labels = test_data['label'].tolist()
test_inputs = tokenizer(test_reviews, padding=True, truncation=True, return_tensors='tf', max_length=128)  # Adjust max_length
test_inputs_numpy = {
    'input_ids': np.array(test_inputs['input_ids']),
    'attention_mask': np.array(test_inputs['attention_mask'])
}
test_labels = tf.convert_to_tensor(test_labels)

In [10]:
# Compile and train the model with a specified batch size (adjust as needed)
batch_size = 32
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_inputs_numpy, train_labels, epochs=3, batch_size=batch_size, validation_data=(test_inputs_numpy, test_labels))



Epoch 1/3
2500/2500 [==============================] - 1054s 411ms/step - loss: 0.6970 - accuracy: 0.6945 - val_loss: 0.6931 - val_accuracy: 0.6996
Epoch 2/3
2500/2500 [==============================] - 1081s 432ms/step - loss: 0.6931 - accuracy: 0.6967 - val_loss: 0.6931 - val_accuracy: 0.6996
Epoch 3/3
2500/2500 [==============================] - 1081s 432ms/step - loss: 0.6931 - accuracy: 0.6967 - val_loss: 0.6931 - val_accuracy: 0.6996


In [11]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_inputs_numpy, test_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

625/625 [==============================] - 86s 138ms/step - loss: 0.6931 - accuracy: 0.6996
Test Accuracy: 69.96%
